In [ ]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.stats

# Apartat (C): Analitzant Dades

1. Quin és el tipus de cada atribut? 
2. Quins atributs tenen una distribució Guassiana?
3. Quin és l'atribut objectiu? Per què?

In [ ]:
# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('Admission_Predict_Ver1.1.csv')
data = dataset.values
labels = dataset.columns.values

1. Quin és el tipus de cada atribut? 

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

<style>
table {float:left}
</style>

| NOM | Tipus de dada | Informació |
| :-- | :-: | :-- |
| Serial No. | Nombre enter (int) | representa la posició de la entrada en ordre, no aporta res |
| GRE Score | Nombre enter (int) | representa la puntuació del estudiant a els Graduate Record Examinations, els valors van desde 260 fins a 340, representant el sistema de puntuació a India |
| TOEFL Score | Nombre enter (int) | representa la puntuació del estudiant a els Test Of English as a Foreign Language, els valors poden anar desde 0 fins a 120 |
| University Rating | Nombre enter (int) | representa la puntuació de la universitat pot variar entre 1 i 5 |
| SOP (Statement of Purpose) | Nombre Decimal (float) | representa la valoración de la qualitat del Statement of Purpose, en valors de 0 a 5 en intervals de 0’5 |
| LOR (Letter of Recommendation) | Nombre Decimal (float) | representa la valoración de la qualitat del Letter of Recommendation, en valors de 0 a 5 en intervals de 0’5 |
| CGPA (Undergraduate GPA) | Nombre Decimal (float) | representa la nota mitja de l'estudiant en els seus estudis undergrad. Va de 0 a 10 |
| Research Experience | Booleà (bool) | representa si l’estudiant ha tingut experiència de recerca anteriorment on no |
| Chance of Admission | Nombre Decimal (float) | mostra la probabilitat de ser admès a la universitat donats tots els altres factors |


2. Quins atributs tenen una distribució Guassiana?

In [ ]:
dataset.hist()

Els atributs que tenen una distribució gaussiana, basant en les grafiques son: GRE Score, TOEFL Score, University Rating, SOP, LOR, CGPA i Chance of Admission.

3. Quin és l'atribut objectiu? Per què?

Chance of Admit [...]

# Apartat (B): Primeres regressions

1. Quin són els atributs més importants per fer una bona predicció?

2. Amb quin atribut s'assoleix un MSE menor?

3. Quina correlació hi ha entre els atributs de la vostra base de dades?

4. Com influeix la normalització en la regressió?

5. Com millora la regressió quan es filtren aquells atributs de les mostres que no contenen informació?

6. Si s'aplica un PCA, a quants components es redueix l'espai? Per què?

In [ ]:
def plan_split(size, precentatges):
    assert(sum(precentatges) <= 1)
    indices = np.arange(size)
    split = np.zeros(size)
    np.random.shuffle(indices)
    used = 0
    for i, p in enumerate(precentatges):
        n = int(np.floor(size * p))
        split[indices[used:n]] = i + 1
        used += n
    return split

def perform_split(X, y, split):
    X_ = []
    y_ = []
    for i in range(2):
        X_.append(X[split == i])
        y_.append(y[split == i])
    return X_, y_
    

X = data[:, [i for i in range(8)]]
y = data[:, 8]
split_info = plan_split(X.shape[0], [.15, .15])

X_, y_ = perform_split(X, y, split_info)

1. Quin són els atributs més importants per fer una bona predicció?

In [ ]:
import seaborn as sns
relacio = sns.pairplot(dataset)

In [ ]:
for i in range(8):
    sns.lmplot(x=labels[i], y=labels[8], data=dataset)

Els millors sòn *GRE Score*, *TOEFL Score* i *CGPA*

2. Amb quin atribut s'assoleix un MSE menor?

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score as R2

def regression(x, y):
    # Creem un objecte de regressió de sklearn
    regr = LinearRegression()

    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)

    # Retornem el model entrenat
    return regr

def MSE(v1, v2):
    return float(((v1 - v2)**2).mean())

def calc_MSE(X, y):
    mse = []
    for i in range(X[0].shape[1]):
        r = regression(X[0][:,[i]], y[0])
        mse.append(MSE(r.predict(X[1][:,[i]]), y[1]))
    return mse

def calc_full(X, y):
    r = regression(X[0], y[0])
    pred = r.predict(X[1])
    return (MSE(pred, y[1]), R2(pred, y[1]))

def show_MSE(X, y, labels, extra=""):
    mse = calc_MSE(X, y)
    for i, val in enumerate(mse):
        print("MSE of ", labels[i], " ", extra, ": ", val, sep="")

In [ ]:
show_MSE(X_, y_, labels)

mse = calc_MSE(X_, y_)
print("\nL'atribut amb MSE menor és", labels[mse.index(min(mse))])

3. Quina correlació hi ha entre els atributs de la vostra base de dades?

In [ ]:
correlacio = dataset.corr()

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)

Comentari [...]

4. Com influeix la normalització en la regressió?

In [ ]:
def normalitzar(X):
    mean = X.mean(0)
    std = X.std(0)
    Xnorm = (X - mean[None, :]) / std[None, :]
    return Xnorm

Xnorm = normalitzar(X)
X_norm, y_norm = perform_split(Xnorm, y, split_info)

In [ ]:
show_MSE(X_norm, y_norm, labels, "normalitzat")

mse_norm = calc_MSE(X_norm, y_norm)
print("\nL'atribut normalitzat amb MSE menor és", labels[mse_norm.index(min(mse_norm))])

In [ ]:
tol = 10**-12
dif = [e - e_norm if abs(e - e_norm) > tol else 0 for e, e_norm in zip(mse, mse_norm)]

for i, val in enumerate(dif):
    print("Diference of MSE for ", labels[i], ": ", val, sep="")

In [ ]:
no_normal = calc_full(X_, y_)
normal = calc_full(X_norm, y_norm)
print("Sense normalitzar tenim MSE =", no_normal[0], "i R^2 =", no_normal[1])
print("Si ho normalitzem tenim MSE =", normal[0], "i R^2 =", normal[1])

No hi ha cap canvi apreciable en cap de les mesures.

5. Com millora la regressió quan es filtren aquells atributs de les mostres que no contenen informació?

In [ ]:
Xinfo = Xnorm[:,[1, 2, 3, 4, 5, 6, 7]]
X_info, y_info = perform_split(Xinfo, y, split_info)

In [ ]:
all_used = calc_full(X_, y_)
only_info = calc_full(X_info, y_info)
print("Amb tots els atributs =", all_used[0], "i R^2 =", all_used[1])
print("Només els que contenen informacio util =", only_info[0], "i R^2 =", only_info[1])

Comentari [...]

6. Si s'aplica un PCA, a quants components es redueix l'espai? Per què?

In [ ]:
from sklearn.decomposition import PCA

XbestPCA = PCA(n_components='mle').fit_transform(Xnorm)
X_bestPCA, y_bestPCA = perform_split(XbestPCA, y, split_info)

print("Espai reduit a", XbestPCA.shape[1], "dimensions.")

In [ ]:
all_used = calc_full(X_, y_)
only_info = calc_full(X_info, y_info)
PCA_used = calc_full(X_bestPCA, y_bestPCA)
print("Amb tots els atributs =", all_used[0], "i R^2 =", all_used[1])
print("Només els que contenen informacio util =", only_info[0], "i R^2 =", only_info[1])
print("Reduit utilitzant PCA =", PCA_used[0], "i R^2 =", PCA_used[1])

# Apartat (A): El descens del gradient  

1. Com influeixen tots els paràmetres en el procés de descens? Quins valors de learning rate convergeixen més ràpid a la solució òptima? Com influeix la inicialització del model en el resultat final? 

2. Quines funcions polinomials (de diferent grau, de diferents combinacions d'atributs, ...) heu escollit per ser apreses amb el vostre descens del gradient? quina ha donat el millor resultat (en error i rapidesa en convergència)?

3. Utilitzeu el regularitzador en la fòrmula de funció de cost i descens del gradient i proveu polinomis de diferent grau. Com afecta el valor del regularitzador?

3. Quina diferència (quantitativa i qualitativa) hi ha entre el vostre regressor i el de la llibreria ?

4. Té sentit el model (polinomial) trobat quan es visualitza sobre les dades? 

5. Ajuda la visualització a identificar aquelles mostres per a les que el regressor obté els pitjors resultats de predicció? 

In [ ]:
# TODO

class Regressor(object):
    def __init__(self, w0, w1, alpha):
        # Inicialitzem w0 i w1 (per ser ampliat amb altres w's)
        self.w0 = w0
        self.w1 = w1
        self.alpha = alpha

        
    def predict(self, x):
        # implementar aqui la funció de prediccio
        pass
    
    def __update(self, hy, y):
        # actualitzar aqui els pesos donada la prediccio (hy) i la y real.
        pass
    
    def train(self, max_iter, epsilon):
        # Entrenar durant max_iter iteracions o fins que la millora sigui inferior a epsilon
        pass